# IDEA: How to transform zero-inflated variables?

https://stats.stackexchange.com/questions/1444/how-should-i-transform-non-negative-data-including-zeros

Given a random variable $x$ supported on $[0,∞)$ such that zero values occur potentially often.

Transform $x⟼\log(x+c)$, with $c>0$, ([Box-Cox-Transformation
](https://de.wikipedia.org/wiki/Box-Cox-Transformation)) idea. How to choose $c$? Many suggestions have been made:

- Half of the smallest non-zero parameter
- Square of the first quartile divided by the third quartile (Statistische Datenanalyse Eine Einführung für Naturwissenschaftler)

We suggest another approach specifically for Machine Learning. The goal is transforming the data in a way that makes it look as Like a standard Normal distribution. (which can be considered as a preconditioning step)

We consider the problem:

$$ α^*, β^*, γ^* =  \operatorname*{arg\,min}_{α>0, β, γ>0} D_{ᴋʟ}(ϕ(p^\text{emp}), 𝓝(0,1))
\qquad ϕ(x) = α⋅\log(x+γ)+β
$$

I.e. we combine the log transform with a simple linear/affine transform.

Note that the parameter restrictions make $ϕ$ strictly monotonically increasing. In this case the transformed density is simply


$$ p_Y = p_X(ϕ^{-1}(y)) \frac{∂ϕ^{-1}(y)}{∂y} $$


Note that the inverse is $ϕ^{-1}(y) = \exp(\frac{y-β}{α}) - γ$. It might be simpler to 


**Alternative:** Consider a linear-log-linear transform $x⟼σ⋅\log(αx+β)+μ$ with $α>0, σ>0, β>0$.
One may simply estimate $μ, σ$ from the data. 

The result of the transforming an empirical distribution is again an empirical distribution.

Note that, given a empirical distribution $p^\text{emp} = \frac{1}{N}\sum_{n=1}^{N}δ(x_n)$, the KL-divergence is

$$ 𝐃_{ᴋʟ}(p^\text{emp}, q) = 𝐇(p^\text{emp}) + \frac{1}{N}\sum_{n=1}^{N}-\log(q(x_n)) $$

Note that the entropy term is infinite, hence minimizing the divergence restricts to the second term. 

## Comparing to Normal Distribution

Given a Gaussian distribution $q=𝓝(μ,σ²)$, we have

$$
ℓ(μ, σ) = \frac{1}{N}\sum_{n=1}^{N}-\log(q(x_n)) 
= \frac{1}{N}\sum_{n=1}^{N} \frac{1}{2}\Big(\log(2πσ^2) + \big(\frac{x_n-μ}{σ}\big)^2 \Big)
$$

Which is of course just the negative log-likelihood of the Gaussian. The optimal choice for $μ$ and $σ$ are hence the estimated mean and standard deviation.

$$\begin{aligned}
   0 &= \frac|{∂ℓ}{∂μ} = \frac{1}{N}\sum_{n=1}^{N}\frac{μ-x_n}{σ^2}
& ⇝ μ &=  \frac{1}{N}\sum_{n=1}^{N}x_n
\\ 0 &= \frac{∂ℓ}{∂σ} = \frac{1}{N}\sum_{n=1}^{N}\Big( \frac{1}{σ} - \frac{(x_n-μ)^2}{σ^3}\Big)
& ⇝ σ^2 &=  \frac{1}{N}\sum_{n=1}^{N}(x_n-μ)^2
\end{aligned}$$

## Normal Distribution with built in transform to standard normal



Given $Y = ψ(X)$, with $X∼𝓝(0,1)$ and $ψ(x)=σx+μ$, then $q(y) = p(ψ^{-1})\frac{∂ψ^{-1}}{∂ψ} = \frac{1}{\sqrt{2πσ^2}}e^{-(\frac{x-μ}{σ})^2}$

$$ 𝐃_{ᴋʟ}(p_y^\text{emp}, )) ∼ \frac{1}{N}∑_{n=1}^{N} \log(p()) $$

## Comparing to Uniform Distribution

Given a Uniform distribution $q=𝓤(a,b)$, we have

$$
ℓ(a, b) = \frac{1}{N}\sum_{n=1}^{N}-\log(q(x_n)) 
= \frac{1}{N}\sum_{n=1}^{N}\log(b-a)
$$

The issue here is that it is independent of the data.

# Idea: Wasserstein distances!


In 1d, we can make use of the fact that 

$$  W_{p}(\mu _{1},\mu _{2})=\left(\int _{0}^{1}\left|F_{1}^{-1}(q)-F_{2}^{-1}(q)\right|^{p}\,\mathrm {d} q\right)^{1/p}
\qquad\text{and}\qquad  W_{1}(\mu _{1},\mu _{2})=\int _{\mathbb {R} }\left|F_{1}(x)-F_{2}(x)\right|\,\mathrm {d} x
$$

Given that $μ_1$ is an empirical distribution, i.e. $F_1 = \frac{1}{N}∑_{n=1}^N H(x-x_n)$, with the Heaviside step function $H$, we have, using $x_0=-∞$ and $x_{N+1}=+∞$. 

WLOG assume $x_1 ≤ x_2 ≤ … ≤ x_n$. The quantile function is a step function as well and can be expressed as

$$F_1^{-1}(q) = \max\{x_n ∣ x_n < q\} = x_1 + ∑_{k≤Nq}(x_{k+1}-x_{k}) = x_1 + ∑_{k=1}^{N-1} (x_{k+1}-x_k) H(q-\tfrac{k}{N}) = x_{\max\{n∣n<Nq\}}$$

Plugging this back into the equation we obtain

$$\begin{aligned}
W_{p}(p^\text{emp}, μ)^p 
&= \int _{0}^{1} \Big| x_1 +\sum_{k=1}^{N-1} (x_{k+1}-x_k) H(q-\tfrac{k}{N})   -F^{-1}(q) \Big|^{p} dq
\\ &=∑_{k=1}^N ∫_{\tfrac{k-1}{N}}^{\tfrac{k}{N}} \Big| x_k  - F^{-1}(q) \Big|^{p} dq
\end{aligned}$$


$$ {\displaystyle W_{1}(\mu _{1},\mu _{2})=\int _{\mathbb {R} }\left|F_{1}(x)-F_{2}(x)\right|\,\mathrm {d} x}  = … = ∑_{n=0}^N ∫_{x_n}^{x_{n+1}} |\tfrac{n}{N} - F(x) | dx $$



$$ W_{p}(\mu _{1},\mu _{2})^p =\int _{0}^{1}\left|F_{1}^{-1}(q)-F_{2}^{-1}(q)\right|^{p}\,\mathrm {d} q
=  ∑_{k=1}^{N} ∫_{\frac{k-1}{N}}^{\frac{k}{N}} |x_k - F^{-1}(q)|^p 𝖽 q
$$

## Case p=2


This case is special because it allows us to get rid of the absolute value:
$$\begin{aligned} W_{2}(\mu _{1},\mu _{2})^2 
&=  ∑_{k=1}^{N} ∫_{\frac{k-1}{N}}^{\frac{k}{N}} |x_k - F^{-1}(q)|^2 𝖽 q
\\ &=  ∑_{k=1}^{N} ∫_{\frac{k-1}{N}}^{\frac{k}{N}} x_k^2 - 2 x_k F^{-1}(q) + F^{-1}(q)^2 𝖽 q
\\ &=  ∑_{k=1}^{N} \Big[  \frac{1}{N}x_k^2 - 2\Big(∫_{\frac{k-1}{N}}^{\frac{k}{N}} F^{-1}(q) dq\Big)x_k   + ∫_{\frac{k-1}{N}}^{\frac{k}{N}}F^{-1}(q)^2 𝖽 q \Big]
\\ &= \frac{1}{N}∑_{k=1}^{N} \Big[ x_k^2 - 2 N \Big(∫_{\frac{k-1}{N}}^{\frac{k}{N}} F^{-1}(q) dq\Big)x_k\Big] + ∫_{0}^{1} F^{-1}(q)^2 𝖽 q
\\ &=\frac{1}{N}∑_{k=1}^{N} \Big[ x_k^2 - 2N \Big(∫_{\frac{k-1}{N}}^{\frac{k}{N}} F^{-1}(q) dq\Big) x_k  + ∫_{0}^{1} F^{-1}(q)^2 𝖽 q \Big]
\end{aligned}$$

## Uniform distribution

Consider the case when q is a uniform distribution. Then the CDF is $\frac{x_{[a,b]}-a}{b-a}$. the quantile function is therefore $F^{-1}(q) = a+(b-a)q$. Thus:


$$\begin{aligned}
∫_{x_{\min}}^{x_{\max}} F^{-1}(q) dq 
&= ∫_{x_{\min}}^{x_{\max}} (a+(b-a)q) dq 
= (a + (b-a)\overline{x}){∆x}
\\ ∫_{x_{\min}}^{x_{\max}} F^{-1}(q)^2 dq 
&= ∫_{x_{\min}}^{x_{\max}} (a+(b-a)q)^2 dq 
= \tfrac{(a+(b-a)x_{\max})^3 - (a+(b-a)x_{\min})^3}{3{∆x}}
\\ ∫_{0}^{1} F^{-1}(q)^2 dq 
&= (a^2 + ab + b^2)/3
\end{aligned}$$

Using this quantile function, we can write down the Wasserstein metric explicitly:

$$\begin{aligned} W_2(p^\text{emp}, U(a,b))^2 
&=\frac{1}{N}∑_{k=1}^{N} \Big[ x_k^2 - 2N \Big(∫_{\frac{k-1}{N}}^{\frac{k}{N}} F^{-1}(q) dq\Big) x_k  + ∫_{0}^{1} F^{-1}(q)^2 𝖽 q \Big]
\\ &= \frac{1}{N}∑_{k=1}^N \Big[ x_k^2
- 2 N\Big(  \big(a + (b-a)\tfrac{2k-1}{2N})\big)\tfrac{1}{N}  \Big) x_k 
+ \tfrac{1}{3}(a^2 + ab + b^2)
\Big]
\end{aligned}$$

In particular, if we choose the mean-0, variance-1 uniform distribution:

$$\begin{aligned} W_2(p^\text{emp}, U(-√3,+√3))^2 
 &= \frac{1}{N}∑_{k=1}^N \Big[ x_k^2
- 2 N\Big(  \big(a + (b-a)\tfrac{2k-1}{2N})\big)\tfrac{1}{N}  \Big) x_k 
+ \tfrac{1}{3}(a^2 + ab + b^2)
\Big]
\\ &= \frac{1}{N}∑_{k=1}^N x_k^2
+\frac{2}{N}∑_{k=1}^N x_k (1 - \tfrac{2k-1}{N})\sqrt{3} + 1
\\ &= \frac{1}{N}∑_{k=1}^N \Big[x_k^2 + 2\sqrt{3}\underbrace{(1 - \tfrac{2k-1}{N})}_{∈(-1, +1)} x_k + 1   \Big]
%\\ &= \frac{1}{N}∑_{k=1}^N \Big[\Big(x_k^2 - 2\sqrt{3}x_k + 3 \Big) + 2\sqrt{3}(2 - \tfrac{2k-1}{N})x_k \Big]
%\\ &= \frac{1}{N}∑_{k=1}^N \Big[\underbrace{(x_k - \sqrt{3})^2}_{≥0} + 2\sqrt{3}(2 - \tfrac{2k-1}{N})x_k\Big]
\end{aligned}$$

**Problem: Need to prove non-negativity:** Each term is of the form $x^2 + 2\sqrt{3}ax + 1$. This expression is guaranteed to be non-negative if and only if $\sqrt{3}a∈[-1, +1]$, which is not guaranteed since $1 - \tfrac{2k-1}{N}$ ranges from $1 - \tfrac{2⋅1-1}{N} = 1 - \tfrac{1}{N} <1$ for $k=1$ to  $1 - \tfrac{2⋅N-1}{N} = -1 + \tfrac{1}{N}>-1$ for $k=N$.


**Solution:** This is actually easy. Notice that the function $f(x) = x^2 - 2ax + 1$ attains its minimum at $x=-a$, $f(x) = 1 - a^2$. Performing this inequality term-by-term and plugging the result back into the equation we have that 


$$\begin{aligned} 
W_2(p^\text{emp}, U(-√3,+√3))^2 
   &≥ \frac{1}{N}∑_{k=1}^N 1 - 3(1 - \tfrac{2k-1}{N})^2
\\ &= 1 - \frac{3}{N} ∑_{k=1}^N  \Big[1 -  \frac{4k-2}{N} + \frac{4k^2 - 4k + 1}{N^2}\Big]
\\ &= 1 - \frac{3}{N} \frac{N^2-1}{3N}
\\ &= \frac{1}{N^2}
\end{aligned}$$

**Note that this inequality is sharp if indeed the $x_k$ are placed exactly at the locations.**

**Problem 3: Can we prove (quasi-) convexity of the objective function?**

## Gaussian Distribution

$\newcommand{\erf}{\operatorname{erf}}$

given $p(x)=𝓝(x ∣ μ, Σ)$, hence $F(x) = \tfrac{1}{2}(1+\erf(\frac{x-μ}{σ\sqrt{2}}))$ and $F^{-1}(q)=μ+σ\sqrt{2}\erf^{-1}(2q-1)$

$$\begin{aligned}
∫_{x_{\min}}^{x_{\max}} F^{-1}(q) dq 
&= ∫_{x_{\min}}^{x_{\max}} μ+σ\sqrt{2}\erf^{-1}(2q-1) dq 
= \frac{1}{\sqrt{2π}} \Big( e^{-\erf^{-1}(2a-1)^2} - e^{-\erf^{-1}(2b-1)^2}   \Big)
\\ ∫_{x_{\min}}^{x_{\max}} F^{-1}(q)^2 dq 
&= ∫_{x_{\min}}^{x_{\max}} \big(μ+σ\sqrt{2}\erf^{-1}(2q-1)\big)^2 dq 
= (b-a) + \frac{1}{\sqrt{π}}\Big( \erf^{-1}(2a-1)e^{-\erf^{-1}(2a-1)^2} - \erf^{-1}(2b-1)e^{-\erf^{-1}(2b-1)^2}   \Big)
\\ ∫_{0}^{1} F^{-1}(q)^2 dq 
&= μ^2 + σ^2
\end{aligned}$$

In [ ]:
import sympy as sp

n, k = sp.symbols("n k")
s = sp.summation((1 - (2 * k - 1) / n) ** 2, (k, 1, n))
s.simplify()

## With duplicate Measurements

Let $μ$ be an empirical distribution $\frac{1}{N}∑_{n=1}^N δ(x-x_n)$. We assume that we have a **multi-set** $\{x_k:μ_k∣k=1…m\}$, i.e. $$\frac{1}{N}∑_{n=1}^N δ(x-x_n) = ∑_k \tfrac{μ_k}{∑_i μ_i} δ(x-x_k) = ∑_k α_k δ(x-x_k)$$

In this case, the CDF is $F(x) = ∑_k α_k H(x-x_k)$ and the consequently, the quantile function is, assuming wlog $x_1 < x_2 < … < x_m$,

$$ F^{-1}(q) = x_1 +  ∑_{k=1}^{m-1} (x_{k+1}-x_k) H(q - ∑_{l=1}^k α_l) = x_1 +  ∑_{k=1}^{m-1} {∆x_k} H(q - a_k)$$

Plugging this back into the $W_2$ we get

$$\begin{aligned}
W_2^2(p^\text{emp}，p) 
   &= ∫_0^1 (G^{-1}(q) - F^{-1}(q))^2 dq
\\ &= ∫_0^1 G^{-1}(q)^2 dq - 2∫_0^1 G^{-1}(q)F^{-1}(q) dq +  ∫_0^1 F^{-1}(q)^2 dq
\end{aligned}$$

Consider the first term
$$\begin{aligned}
∫_0^1 G^{-1}(q)^2 dq
 &= ∫_0^1 (x_1 +  ∑_{k=1}^{m-1} {∆x_k} H(q - a_k))^2 dq
\\ &=
∑_{l=1}^m ∫_{a_{l-1}}^{a_l}  (x_1 +  ∑_{k=1}^{m-1} {∆x_k} H(q - a_k))^2 dq
\\ &= ∑_{l=1}^m ∫_{a_{l-1}}^{a_l}  x_l^2 dq
\\ &= ∑_{l=1}^m α_l x_l^2 dq
\end{aligned}$$



Consider the second term
$$\begin{aligned}
∫_0^1 G^{-1}(q)F^{-1}(q) dq
 &= ∫_0^1 (x_1 +  ∑_{k=1}^{m-1} {∆x_k} H(q - a_k))F^{-1}(q) dq
\\ &=
∑_{l=1}^m ∫_{a_{l-1}}^{a_l}  (x_1 +  ∑_{k=1}^{m-1} {∆x_k} H(q - a_k))F^{-1}(q) dq
\\ &= ∑_{l=1}^m ∫_{a_{l-1}}^{a_l}  x_l F^{-1}(q) dq
\\ &= ∑_{l=1}^m \Big(∫_{a_{l-1}}^{a_l}  F^{-1}(q) dq\Big) x_l
\\ &= ∑_{l=1}^m β_l x_l
\end{aligned}$$

Consider the third term
$$\begin{aligned}
∫_0^1 F^{-1}(q)^2 dq
= ∑_{l=1}^m α_l ∫_0^1 F^{-1}(q)^2 dq
= ∑_{l=1}^m α_l C
\end{aligned}$$

Combining these results we obtain

$$W_2^2(p^\text{emp}，p) = ∑_{l=1}^m \Big[ α_l x_l^2  -2 β_l x_l + α_l C \Big] 
\qquad 
$$

$$W_2^2(p^\text{emp}，p) = ∑_{l=1}^m \Big[ α_l x_l^2  -2 β_l x_l + α_l C \Big] 
\qquad αₖ = \frac{μ_k}{∑_{i}μ_i} \qquad  β_k =  \int_{a_{k-1}}^{a_k} F^{-1}(q)dq \qquad C = \int_0^1 F^{-1}(q)^2 dq \qquad γ_k = α_k C
$$

## Case p: Uniform $[a,b]$

In this case the quantile function is the very simple $F^{-1}(q) = a + (b-a)q$.

Then:

- $∫_{x_{\min}}^{x_{\max}} F^{-1}(q) dq = \big(a + (b-a)\frac{x_{\max} + x_{\min}}{2}\big)(x_{\max} - x_{\min})$
- Thus $β_l= \big(a + (b-a)\frac{a_{l} + a_{l-1}}{2}\big) α_l$
- $C = ∫_{0}^{q} F^{-1}(q)^2 dq = \tfrac{1}{3}(a^2 + ab + b^2)$

In particular, when $a=-\sqrt{3}$ and $b=+\sqrt{3}$:

- $C = \tfrac{1}{3}(3 -3 + 3) = 1$
- $β_k = \big(-\sqrt{3} + (\sqrt{3}-(-\sqrt{3}))\frac{a_{l} + a_{l-1}}{2}\big) α_k = -α_k\sqrt{3}(1- (a_{k} + a_{k-1}))$

Plugging these back in:

$$\begin{aligned}
W_2^2(p^\text{emp}，p) 
   &= ∑_{k=1}^m \Big[ α_k x_k^2  -2 β_k x_k + α_k C \Big]
\\ &= ∑_{k=1}^m \Big[ α_k x_k^2  +2 α_k\sqrt{3}(1- (a_{k} + a_{k-1})) x_k + α_k  \Big]
\\ &= ∑_{k=1}^m α_k\Big[  x_k^2  + 2 \sqrt{3}\underbrace{(1- (a_{k} + a_{k-1}))}_{∈(-1, +1)} x_k + 1 \Big]
\\ &= ∑_{k=1}^m α_k\Big[  x_k^2  + 2 \sqrt{3}γ_k x_k + 1 \Big]
\end{aligned}$$

Here, $x_1<…<x_m$ are arbitrary real values, $α_k$ are positive weights that sum to 1. $a_k = ∑_{j=1}^k α_j$

$$ γ_k = 1-a_k - a_{k-1} = ∑_{j=1}^{n} ε_{jk}α_j$$

Individually, each term becomes minimal if $x_k = -a = -√{3}γ_k$, in which case the minimum attained is $1-a^2 = 1-3γ_k^2$


$$\begin{aligned}
W_2^2(p^\text{emp}，p) 
   &= ∑_{k=1}^m α_k\Big[  x_k^2  + 2 \sqrt{3}γ_k x_k + 1 \Big]
\\ &≥ ∑_{k=1}^m α_k (1-3γ_k^2)
\\ &= 1 - 3 ∑_{k=1}^m α_k γ_k^2
\\ &= 1 - 3 ∑_{k=1}^m α_k \big(1 - α_k - 2∑_{j=1}^{k-1}α_j\big)^2
\\ &= 1 - 3 ∑_{k=1}^m α_k \big(∑_{j=k+1}^{n}α_j - ∑_{j=1}^{k-1}α_j\big)^2
\\ &= 1 - 3 ∑_{k=1}^m α_k \big(∑_{j=1}^{n}ε_{jk}α_j\big)^2 
\qquadε_{jk}=\begin{cases}+1:&j<k\\\hfill 0:&j=k\\-1:&j>k\end{cases}
\\ &= 1 - 3 ∑_{k=1}^m α_k ∑_{ij}ε_{ik}ε_{jk}α_iα_j
\\ &= 1 - 3 ∑_{ijk}ε_{ik}ε_{jk} α_iα_j α_k 
\\ &= 1 - 3 ⟨E∣α^{⊗3}⟩
\qquad E_{ijk}=\begin{cases}+1:& (i<k∧j<k) ∨ (i>k∧j>k) \\\hfill 0:& i=k ∨ j=k & \\-1:& (i>k∧j<k)∨ (i<k∧j>k)\end{cases}
\end{aligned}$$

The very nice property of representing the sum this way is that now we have


$$ \frac{d}{dα_k} ∑_{k} α_k \big(∑_{j=1}^{n}ε_{jk}α_j\big)^2 = \big(∑_{j=1}^{n}ε_{jk}α_j\big)^2
$$

since $ε_{kk}=0$. Consequently, setting the gradient equal to zero we have

$$ 0 \overset{!}{=} \big(∑_{j=1}^{n}ε_{jk}α_j\big)^2 
⟺ 0 = ∑_{j=1}^{n}ε_{jk}α_j \quad\text{for all k}
$$

This has $α=0$ as a solution which is not allowed. However, we can instead consider the Lagrangian of the constrained problem

$$ \max_α ∑_{k} α_k \big(∑_{j=1}^{n}ε_{jk}α_j\big)^2 \qquad α≥0 \qquad ∑ α_k = 1$$

**IDEA: SHOW PERMUTATION INVERIANCE + (SCHUR-) CONVEXITY ⟹ OPTIMUM ON DIAGONAL**

Then

$$ 𝓛(α, λ, μ): -∇f(𝛂) + λ⋅𝟏 - 𝛍 = 𝟎 \qquad 𝛍≥0$$

In particular, by multiplying with $𝟏^⊤$ we obtain:

$$λ = \frac{1}{n}∑μ_i + \frac{1}{n}1^⊤∇f(𝛂)$$

Hence

$$ ∇f(𝛂)- \overline{∇f(𝛂)}𝟏 = \boldsymbol{\mu} - \overline{\boldsymbol{\mu}}𝟏 $$

$$ -\Big(∑_{j=1}^{n}ε_{jk}α_j\Big)^2 + λ- μ_k = 0 \qquad\text{for all k}$$

Show: $α_k = \frac{1}{n}$. In this case:$\Big(∑_{j=1}^{n}ε_{jk}α_j\Big)^2 = \frac{((n-1)-2k)^2}{n^2} $ 

In [ ]:
import numpy as np


def E(n):
    E = np.ones((n, n), dtype=int)
    return np.triu(E) - np.triu(E).T


E(9)

In [ ]:
np.linalg.matrix_rank(E(9))

The latter sum we can again maximize over $α∈Δ^{n-1}$.

## Combining with log-transform

Assume we have $y = ψ(x) = \log(x+c)$ with $c>0$ and are given $x∼p^\text{emp}$ with $p^\text{emp}(x) = \tfrac{1}{N}∑δ(x-x_n)$

Note that $x=ψ^{-1}(y) = e^{y} - c$ with $\frac{∂ψ^{-1} }{∂y} = e^{y}$

Then the transformed density has the distribution

$$p(y) = p^\text{emp}(e^{y}-c)|𝐃ψ^{-1}(y)| = e^y \tfrac{1}{N}∑δ(ψ^{-1}(y)-x_n)$$

Note that $ψ^{-1}(y)-x_n = 0$ if and only if $y = ψ(x_n) = y_n$. Note that $δ(g(x)) = ∑_{z: g(z)=0} \frac{δ(x-z)}{|g'(z)|}$, hence, using $g(y) = ψ^{-1}(y)-x_n)$, which has a single root at $y_n=ψ(x_n)$, we have $δ(g(y)) = \frac{δ(y-y_n)}{|g'(y_n)|} = e^{-y_n} δ(y-y_n)$

Plugging this back into the equation we have:

$$p(y) = p^\text{emp}(e^{y}-c)|𝐃ψ^{-1}(y)| = \tfrac{1}{N}∑ e^{y-y_n} δ(y-y_n)$$

Note that since $δ(y-y_n) = 0$ if $y≠y_n$, this distribution is equivalent to

$$p(y) = \tfrac{1}{N}∑ e^{y_n-y_n} δ(y-y_n) = \tfrac{1}{N}∑ δ(y-y_n)$$

I.e. simply the empirical distribution of the $y_n$.

### Plugging into the Wasserstein formula

Assuming $x_1<…<x_n$, then also $y_1<…<y_n$, with $y_k = \log(x_n + c)$. Then

$$\begin{aligned} W_2(p^\text{emp}, U(-√3,+√3))^2 
&= ∑_{k=1}^m α_k\Big[  \log(x_k + c)^2  + 2 \sqrt{3}γ_k \log(x_k + c) + 3 \Big]
\\ &= ∑_{k=1}^m α_k\Big[ y_k^2  + 2 \sqrt{3}γ_k y_k + 3 \Big]
\\ ⟹ ∇ℓ(c) &= 2∑_{k=1}^m α_k\frac{\log(x_k + c)+\sqrt{3}γ_k}{x_k+c}
\\ &= 2∑_{k=1}^m α_k\frac{y_k+\sqrt{3}γ_k}{e^{y_k}}
\\ ⟹ ∇²ℓ(c) &= 2∑_{k=1}^m α_k \frac{1 -\log(x_k + c)-\sqrt{3}γ_k}{(x_k+c)^2}
\\ &= 2∑_{k=1}^m α_k \frac{1 -y_k-\sqrt{3}γ_k}{e^{2y_k}}
\end{aligned}$$

**Theorem:** If there exists a $ω≥0$ such that $𝐌(x, ω) ≔ ∇²ℓ(x) + ω∇ℓ(x)∇ℓ(x)^⊤$ is non-negative for all $x$, then $f(x)$ is pseudoconvex.

### Case n=1

$$\begin{aligned}
𝐌(c, ω) &=  ∇²ℓ(c) + ω∇ℓ(c)∇ℓ(c)^⊤
\\ &= 2\frac{1 -\log(x + c)-\sqrt{3}γ}{(x+c)^2} + ω \Big(2\frac{\log(x + c)+\sqrt{3}γ}{x+c}\Big)^2
\\ ⟹ 0\overset{!}{=} ∇_ω 𝐌(c, ω) &= 2\Big(\frac{\log(x + c)+\sqrt{3}γ}{x+c}\Big)^2
\\ ⟹ c &= e^{\sqrt{3}γ} - x
\\ ⟹ 𝐌(e^{\sqrt{3}γ} - x, ω) &=  \frac{2 -4 \sqrt{3}γ}{e^{2\sqrt{3}γ}} + ω \frac{48γ^2}{e^{2\sqrt{3}γ}   }
\\ &=  2\frac{1 -2 \sqrt{3}γ + ω24γ^2}{e^{2\sqrt{3}γ}}
%\\ &= \frac{1-\sqrt{3}γ+3ωγ^2  + (2ω\sqrt{3}γ -1 )\log(x + c) + ω\log(x + c)^2  }{\frac{1}{2}(x+c)^2}
%\\ &= \frac{1-\sqrt{3}γ + 3ωγ^2  + (2ω\sqrt{3}γ -1 )\log(x + c) + ω\log(x + c)^2  }{\frac{1}{2}(x+c)^2}
\end{aligned}$$

Which is indeed guaranteed non-negative if $ω$ is sufficiently large.


### Case n

$$\begin{aligned}
𝐌(c, ω) &=  ∇²ℓ(c) + ω∇ℓ(c)∇ℓ(c)^⊤
\\ &=  ∑_{k=1}^m 2 α_k \frac{1 -y_k-\sqrt{3}γ_k}{e^{2y_k}} 
      + ω\Big(∑_{k=1}^m 2 α_k\frac{y_k+\sqrt{3}γ_k}{e^{y_k}} \Big)^2
\\ &≥  ∑_{k=1}^m \frac{2α_k}{e^{2y_k}}\Big(1 -y_k-\sqrt{3}γ_k + 2α_kω(y_k+\sqrt{3}γ_k)^2\Big)
%
\\ &≥  ∑_{k=1}^m \frac{2α_k}{e^{2y_k}}\Big[
            \underbrace{\big(\tfrac{1}{4} -\sqrt{3}γ_k + 3γ_k^2\big)}_{\text{$≥0$}} 
            + \underbrace{3(2αₖω-1)γ_k^2}_{\text{$≥0$ if $ω≥\tfrac{1}{2α_k}$}}
          + \big(\tfrac{3}{4} - (1 - 4\sqrt{3}γ_kα_k ω)y_k +  2α_kωy_k^2 \big)\Big]
\\ &≥  ∑_{k=1}^m \frac{2α_k}{e^{2y_k}}\Big[
            \underbrace{\big(\tfrac{1}{2} -\sqrt{3}γ_k\big)^2}_{\text{$≥0$}} 
            + \underbrace{3(2αₖω-1)γ_k^2}_{\text{$≥0$ if $ω≥\tfrac{1}{2α_k}$}}
            + \underbrace{\big(\tfrac{1}{4} -y_k + y_k^2 \big)}_{≥0}
          + \big(\tfrac{1}{2} + 4\sqrt{3}γ_kα_k ω y_k +  (2α_kω-1)y_k^2 \big)\Big]
\\ &≥  ∑_{k=1}^m \frac{2α_k}{e^{2y_k}}\Big[
    \underbrace{\big(\tfrac{1}{2} -\sqrt{3}γ_k\big)^2}_{\text{$≥0$}} 
    + \underbrace{3(2αₖω-1)γ_k^2}_{\text{$≥0$ if $ω≥\tfrac{1}{2α_k}$}}
    + \underbrace{\big(\tfrac{1}{2} -y_k\big)^2}_{≥0}
    + \underbrace{\big(\tfrac{1}{2} + 4\sqrt{3}γ_kα_k ω y_k + 192γ_k^2α_k^2ω^2 y_k^2\big)}_{≥0}
    +  (2α_kω-1)y_k^2 - 192γ_k^2α_k^2ω^2 y_k^2 \Big]
\\ &≥  ∑_{k=1}^m \frac{2α_k}{e^{2y_k}}\Big[
    \underbrace{\big(\tfrac{1}{2} -\sqrt{3}γ_k\big)^2}_{\text{$≥0$}} 
    + \underbrace{\big(\tfrac{1}{2} -y_k\big)^2}_{≥0}
    + \underbrace{\big(\tfrac{1}{4} + 8\sqrt{3}γ_kα_k ω y_k\big)^2}_{≥0}
    + \underbrace{3(2αₖω-1)γ_k^2}_{\text{$≥0$ if $ω≥\tfrac{1}{2α_k}$}}
    +  (- 192γ_k^2α_k^2ω^2 + 2α_kω-1)y_k^2 \Big]
%\\ ⟹  0\overset{!}{=} ∇_ω 𝐌(c, ω) &= ∑_{k=1}^m \frac{2 α_k (\log(x_k + c)+\sqrt{3}γ_k)^2}{(x_k + c)^2}
%\\ &=   ∑_{k=1}^m α_k \frac{1}{(x_k+c)^2}
%       - ∑_{k=1}^m α_k \frac{\log(x_k + c)+\sqrt{3}γ_k}{(x_k+c)^2} 
%       + 2ω\Big(∑_{k=1}^m α_k\frac{\log(x_k + c)+\sqrt{3}γ_k}{x_k+c} \Big)^2
\end{aligned}$$

Solve the min-max problem

$$ \max_{ω}\min_{y_k} L(ω, y_k) ≔ 1-(1-4\sqrt{3}α_kγ_kω)y_k + 2α_kωy_k^2 $$


The minimum is obtained when

$$\begin{aligned}
0 = ∇_{y_k}L(ω, y_k) = -(1-4\sqrt{3}α_kγ_kω) + 4α_k ω y_k 
⟺ y_k = \frac{1-4\sqrt{3}α_kγ_kω}{4α_k ω}
\end{aligned}$$

Hence

$$\begin{aligned}
L(ω, y_k^*) &= 1 - (1-4\sqrt{3}α_kγ_kω)\frac{1-4\sqrt{3}α_kγ_kω}{4α_k ω} + 2α_kω\Big(\frac{1-4\sqrt{3}α_kγ_kω}{4α_k ω}\Big)^2
\\ &= 1 - \frac{(1-4\sqrt{3}α_kγ_kω)^2}{4α_k ω} +\frac{1}{2}\frac{(1-4\sqrt{3}α_kγ_kω)^2}{4α_k ω}
\\ &= 1 - \frac{(1-4\sqrt{3}α_kγ_kω)^2}{8α_k ω}
\end{aligned}$$

#### Maximization step

We need $L(ω, y_k^*) ≥0$, which is equivalently:


$$\begin{aligned}
1 - \frac{(1-4\sqrt{3}α_kγ_kω)^2}{8α_k ω} &≥ 0 
\\ ⟺ 8α_k ω - (1-4\sqrt{3}α_kγ_kω)^2 &≥ 0
\\ ⟺ 48α_k^2γ_k^2ω^2 - 8α_k(1-\sqrt{3}γ_k)ω + 1 &≤ 0 
\end{aligned}$$

#### Maximization step

$$ 0 = ∇_ω L(ω, y_k^*) = \frac{1-48α_k^2γ_k^2ω^2}{8α_kω^2}
⟺ ω = ±\frac{1}{4\sqrt{3}α_k |γ_k|}
$$

Optimal value

The last term is negative for too large $ω$. Its maximum is attained when

$$ 0\overset{!}{=} ∇_ω- 192γ_k^2α_k^2ω^2 + 2α_kω-1

We need to show that there exists a $ω≥0$ such that $𝐌(c, ω)≥0$ for all $c$.

Note that, for each term:

$$
2\frac{\log(x_k + c)}{x+c}  + 2 \frac{\sqrt{3}γ_k}{x_k + c} 
= 0 ⟺ \log(x_k + c) = \sqrt{3}γ_k
$$

For a single term, the derivative

# Numerical Calculations

In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from jax.config import config

config.update("jax_enable_x64", True)
config.update("jax_platform_name", "cpu")
import jax.numpy as np
from jax import grad, jacfwd, vmap
from numpy import pi as π

In [ ]:
from tsdm.datasets import KIWI_RUNS

dataset = KIWI_RUNS()

ts = dataset.timeseries

data = np.array(ts.Glucose[pd.notna(ts.Glucose)].astype(float))
data

In [ ]:
def wasserstein_uniform(x):
    N = len(x)
    n = np.arange(1, N + 1)
    r = x**2 + 2 * np.sqrt(3) * (1 - (2 * n - 1) / N) * x + 3
    return np.mean(r)


def loss(p):
    return -np.mean(np.log(p))


def pdf_gaussian(x, μ=0.0, σ=1.0):
    return np.exp(-(((x - μ) / σ) ** 2) / 2) / np.sqrt(2 * π * σ**2)


def pdf_uniform(x, a=-np.sqrt(3), b=+np.sqrt(3)):
    m = (x >= a) & (x <= b)
    return np.where(m, 1 / (b - a), 0)


def h_uniform(lb=-np.sqrt(3), ub=+np.sqrt(3)):
    return np.log(ub - lb)


def h_gaussian(μ=0.0, σ=1.0):
    return 0.5 + np.log(2 * π * σ**2) / 2

In [ ]:
def model(x, θ):
    x = np.sort(x)
    a, b = θ
    y = np.log(a * x + b)
    # return y
    μ = np.mean(y)
    σ = np.std(y)
    return (y - μ) / σ

In [ ]:
%matplotlib inline
from scipy.optimize import LinearConstraint, minimize

constr = LinearConstraint(np.eye(2), 0, ub=np.inf)


def f(θ):
    return wasserstein_uniform(model(data, θ))


x0 = np.array([1.0, 1.0])
sol = minimize(
    f, x0, method="Nelder-Mead", jac=grad(f), bounds=[(0, np.inf), (0, np.inf)]
)
sol.x, sol.fun, sol.success, sol.status

## Plot the values

In [ ]:
# a = np.linspace(0, 10, 1024)
# b = np.linspace(0, 10, 1024)
a = np.logspace(-6, 2, 512)
b = np.logspace(-6, 2, 512)
A, B = np.meshgrid(a, b)

In [ ]:
C = np.stack([A, B], axis=-1)
Z = vmap(vmap(f))(C)  # this can take a bit of time
np.nanmin(Z), np.nanmax(Z), np.isnan(Z).mean()

In [ ]:
zmin = np.nanmin(Z)
zmax = np.nanmax(Z)
C = -np.log((Z - zmin) / (zmax - zmin))
vmin = 0
vmax = C[np.isfinite(C)].max()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
im = ax.contourf(
    A, B, C, vmin=vmin, vmax=vmax, levels=100, antialiased=True, cmap="plasma"
)
ax.set_yscale("log")
ax.set_xscale("log")
ax.set_aspect("equal", "box")
ax.set_xlabel("a")
ax.set_ylabel("b")
fig.colorbar(im)
fig.savefig("solution.png")

## Conclusion

We can clearly see a straight line in the log-log plot, notifying the existence of a power law:

$$ \log(b) = m⋅\log(a) + c ⟺  b = c⋅a^m $$

This signifies redundancy in our parametrization:

$$  \log(a x + b)  = \log(ax + ca^m) = \log(a(x+ca^{m-1})) = \log(a) + \log(x+ca^{m-1})$$

In essence, it will be enough to do either $\log(x+c)$ or $\log(1+αx)$. Note that both approaches are equivalent since

$$\log(x+c) = \log(c(1+\tfrac{1}{c}x)) = \log(c) + \log(1+\tfrac{1}{c}x)$$

# 1D apporach

In [ ]:
def model_a(x, θ):
    x = np.sort(x)
    y = np.log(1 + θ * x)
    μ = np.mean(y)
    σ = np.std(y)
    return (y - μ) / σ


def model_c(x, θ):
    x = np.sort(x)
    y = np.log(x + θ)
    μ = np.mean(y)
    σ = np.std(y)
    return (y - μ) / σ

In [ ]:
%matplotlib inline
# from jax.scipy.optimize import minimize
from scipy.optimize import minimize


def f(θ):
    return wasserstein_uniform(model_c(data, θ))


jac = grad(f)
hess = jacfwd(jac)

In [ ]:
hess(0.1)

In [ ]:
x0 = np.array([1.0])
sol = minimize(
    f,
    x0,
    method="trust-constr",
    jac=jac,
    hess=hess,
    bounds=[(0, np.inf)],
    options={"disp": True},
)
sol.x, sol.fun, sol.success, sol.status

In [ ]:
t = np.logspace(-6, 6, 1024)
z = vmap(f)(t)
plt.loglog(t, z)

In [ ]:
values = [
    data[data > 0].min() / 2,
    np.quantile(data, 0.25) ** 2 / np.quantile(data, 0.75),
    np.quantile(data, 0.25) ** 2 / np.quantile(data, 0.75) ** 2,
    sol.x,
]

In [ ]:
%matplotlib inline

from scipy.stats import uniform

fig, axes = plt.subplots(
    ncols=len(values),
    constrained_layout=True,
    figsize=(3 * len(values), 3),
    sharey=True,
    sharex=True,
)

t = np.linspace(-3, +3, 1024)
for val, ax in zip(values, axes):
    ax.hist(model_c(data, val), density=True, bins=20)
    ax.plot(t, uniform.pdf(t, loc=-np.sqrt(3), scale=2 * np.sqrt(3)))
    ax.set_yscale("log")

# Observation

- distribution skewed if not normalized
- values at center are positive, values at outside negative as expected
- idea: combine pairs x[1] and x[1/2]

In [ ]:
# data
y = np.log(np.sort(data) + 0.071)
y = (y - y.mean()) / y.std()
N = len(y)
n = np.arange(1, N + 1)
r = y**2 + 2 * np.sqrt(3) * (1 - (2 * n - 1) / N) * y + 1
plt.plot(n, r), np.mean(r)

In [ ]:
um = np.arange(N) >= N // 2
lm = np.arange(N) < N // 2

uy = r.copy()
uy = uy.at[um].set(0)
uy = uy.at[lm].set(y[lm][::-1])
ly = r.copy()
ly = ly.at[lm].set(0)
ly = ly.at[um].set(y[um][::-1])


plt.plot(n, uy + ly)